In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

from scipy import stats
import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh
import gbd_mapping
from vivarium import Artifact

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
# import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as op # for output processing
import model_validation.ciff_sam_results as vr # for Vivarium Results
import model_validation.ciff_sam_plots as csp

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/model_validation/model4
ndbs
Thu Oct  7 15:45:30 PDT 2021


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data1 = vr.VivariumResults.cleaned_from_model_spec('4.5.2')
data1.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'cause_state_person_time',
 'cause_transition_count']

In [4]:
data = vr.VivariumResults.cleaned_from_model_spec('4.5.2')
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'cause_state_person_time',
 'cause_transition_count',
 'person_time']

In [5]:
data.person_time

,age,input_draw,measure,scenario,sex,sq_lns,wasting_treatment,x_factor,x_factor_effect,year,value
0,1-5_months,29,person_time,baseline,female,covered,covered,cat1,1.1,2022,0.000000
1,1-5_months,29,person_time,baseline,female,covered,covered,cat1,1.1,2023,0.000000
2,1-5_months,29,person_time,baseline,female,covered,covered,cat1,1.1,2024,0.000000
3,1-5_months,29,person_time,baseline,female,covered,covered,cat1,1.1,2025,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
86396,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,cat2,1.5,2023,50.591376
86397,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,cat2,1.5,2024,52.776181
86398,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,cat2,1.5,2025,52.197125
86399,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,cat2,1.5,2026,49.886379


In [6]:
data.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,x_factor_effect,value,x_factor,sq_lns,wasting_treatment,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat2,covered,covered,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat1,covered,covered,early_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat2,covered,uncovered,early_neonatal
3,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat1,covered,uncovered,early_neonatal
...,...,...,...,...,...,...,...,...,...,...,...,...
345596,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,16475.270363,cat2,uncovered,covered,2_to_4
345597,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,3419.826146,cat1,uncovered,covered,2_to_4
345598,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,1704.976044,cat2,uncovered,uncovered,2_to_4
345599,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,384.646133,cat1,uncovered,uncovered,2_to_4


In [7]:
data.stunting_state_person_time

,sex,year,stunting_state,measure,input_draw,scenario,x_factor_effect,value,sq_lns,age
0,female,2022,cat1,state_person_time,29,baseline,1.1,0.000000,covered,early_neonatal
1,female,2022,cat1,state_person_time,29,baseline,1.1,0.000000,uncovered,early_neonatal
2,female,2022,cat1,state_person_time,29,baseline,1.1,0.000000,covered,late_neonatal
3,female,2022,cat1,state_person_time,29,baseline,1.1,0.000000,uncovered,late_neonatal
...,...,...,...,...,...,...,...,...,...,...
86396,male,2026,cat4,state_person_time,946,wasting_treatment,1.5,0.000000,covered,12_to_23_months
86397,male,2026,cat4,state_person_time,946,wasting_treatment,1.5,4807.859001,uncovered,12_to_23_months
86398,male,2026,cat4,state_person_time,946,wasting_treatment,1.5,0.000000,covered,2_to_4
86399,male,2026,cat4,state_person_time,946,wasting_treatment,1.5,12216.184805,uncovered,2_to_4


In [8]:
sqlns_coverage_by_age_year = vr.get_sqlns_coverage(data, ['age', 'year'])
sqlns_coverage_by_age_year

,age,year,input_draw,scenario,sq_lns,value,numerator_measure,denominator_measure,multiplier
0,1-5_months,2022,29,baseline,covered,0.0,state_person_time,person_time,1
1,1-5_months,2022,29,baseline,uncovered,1.0,state_person_time,person_time,1
2,1-5_months,2022,29,sqlns,covered,0.0,state_person_time,person_time,1
3,1-5_months,2022,29,sqlns,uncovered,1.0,state_person_time,person_time,1
...,...,...,...,...,...,...,...,...,...
2156,late_neonatal,2026,946,sqlns,covered,0.0,state_person_time,person_time,1
2157,late_neonatal,2026,946,sqlns,uncovered,1.0,state_person_time,person_time,1
2158,late_neonatal,2026,946,wasting_treatment,covered,0.0,state_person_time,person_time,1
2159,late_neonatal,2026,946,wasting_treatment,uncovered,1.0,state_person_time,person_time,1


In [11]:
data.cause_state_person_time.cause_state.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'susceptible_to_diarrheal_diseases',
       'susceptible_to_lower_respiratory_infections',
       'susceptible_to_measles'], dtype=object)

In [13]:
data.deaths.cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'moderate_acute_malnutrition', 'other_causes',
       'severe_acute_malnutrition'], dtype=object)

In [15]:
vr.get_prevalence(data, 'wasting', ['year', 'x_factor'])

,year,x_factor,input_draw,scenario,wasting_state,value,numerator_measure,denominator_measure,multiplier
0,2022,cat1,29,baseline,mild_child_wasting,0.210160,state_person_time,person_time,1
1,2022,cat1,29,baseline,moderate_acute_malnutrition,0.092124,state_person_time,person_time,1
2,2022,cat1,29,baseline,severe_acute_malnutrition,0.025765,state_person_time,person_time,1
3,2022,cat1,29,baseline,susceptible_to_child_wasting,0.671951,state_person_time,person_time,1
...,...,...,...,...,...,...,...,...,...
1436,2026,cat2,946,wasting_treatment,mild_child_wasting,0.218086,state_person_time,person_time,1
1437,2026,cat2,946,wasting_treatment,moderate_acute_malnutrition,0.065728,state_person_time,person_time,1
1438,2026,cat2,946,wasting_treatment,severe_acute_malnutrition,0.014491,state_person_time,person_time,1
1439,2026,cat2,946,wasting_treatment,susceptible_to_child_wasting,0.701696,state_person_time,person_time,1


In [17]:
vr.get_prevalence(data, 'wasting', ['year', 'x_factor'], numerator_broadcast='sq_lns')

,year,x_factor,input_draw,scenario,wasting_state,sq_lns,value,numerator_measure,denominator_measure,multiplier
0,2022,cat1,29,baseline,mild_child_wasting,covered,0.000000,state_person_time,person_time,1
1,2022,cat1,29,baseline,mild_child_wasting,uncovered,0.210160,state_person_time,person_time,1
2,2022,cat1,29,baseline,moderate_acute_malnutrition,covered,0.000000,state_person_time,person_time,1
3,2022,cat1,29,baseline,moderate_acute_malnutrition,uncovered,0.092124,state_person_time,person_time,1
...,...,...,...,...,...,...,...,...,...,...
2876,2026,cat2,946,wasting_treatment,severe_acute_malnutrition,covered,0.000000,state_person_time,person_time,1
2877,2026,cat2,946,wasting_treatment,severe_acute_malnutrition,uncovered,0.014491,state_person_time,person_time,1
2878,2026,cat2,946,wasting_treatment,susceptible_to_child_wasting,covered,0.000000,state_person_time,person_time,1
2879,2026,cat2,946,wasting_treatment,susceptible_to_child_wasting,uncovered,0.701696,state_person_time,person_time,1


In [23]:
ordered_scenarios = ['baseline', 'wasting_treatment', 'sqlns']
ordered_wasting_states = [
    'severe_acute_malnutrition',
    'moderate_acute_malnutrition',
    'mild_child_wasting',
    'susceptible_to_child_wasting',
]
df = vr.column_to_ordered_categorical(data.stunting_state_person_time, 'scenario', ordered_scenarios)
df

,sex,year,stunting_state,measure,input_draw,scenario,x_factor_effect,value,sq_lns,age
0,female,2022,cat1,state_person_time,29,baseline,1.1,0.000000,covered,early_neonatal
1,female,2022,cat1,state_person_time,29,baseline,1.1,0.000000,uncovered,early_neonatal
2,female,2022,cat1,state_person_time,29,baseline,1.1,0.000000,covered,late_neonatal
3,female,2022,cat1,state_person_time,29,baseline,1.1,0.000000,uncovered,late_neonatal
...,...,...,...,...,...,...,...,...,...,...
86396,male,2026,cat4,state_person_time,946,wasting_treatment,1.5,0.000000,covered,12_to_23_months
86397,male,2026,cat4,state_person_time,946,wasting_treatment,1.5,4807.859001,uncovered,12_to_23_months
86398,male,2026,cat4,state_person_time,946,wasting_treatment,1.5,0.000000,covered,2_to_4
86399,male,2026,cat4,state_person_time,946,wasting_treatment,1.5,12216.184805,uncovered,2_to_4


In [24]:
df.dtypes

sex                 object
year                object
stunting_state      object
measure             object
                    ...   
x_factor_effect    float64
value              float64
sq_lns              object
age                 object
Length: 10, dtype: object

In [25]:
df.scenario

0                 baseline
1                 baseline
2                 baseline
3                 baseline
               ...        
86396    wasting_treatment
86397    wasting_treatment
86398    wasting_treatment
86399    wasting_treatment
Name: scenario, Length: 86400, dtype: category
Categories (3, object): ['baseline' < 'wasting_treatment' < 'sqlns']

In [26]:
df1 = vr.to_ordered_categoricals(data.wasting_state_person_time)
df1.dtypes

sex                    object
year                   object
wasting_state        category
measure                object
                       ...   
x_factor               object
sq_lns                 object
wasting_treatment      object
age                  category
Length: 12, dtype: object

In [27]:
df1.wasting_state

0                   mild_child_wasting
1                   mild_child_wasting
2                   mild_child_wasting
3                   mild_child_wasting
                      ...             
345596    susceptible_to_child_wasting
345597    susceptible_to_child_wasting
345598    susceptible_to_child_wasting
345599    susceptible_to_child_wasting
Name: wasting_state, Length: 345600, dtype: category
Categories (4, object): ['severe_acute_malnutrition' < 'moderate_acute_malnutrition' < 'mild_child_wasting' < 'susceptible_to_child_wasting']

In [28]:
df1.scenario

0                  baseline
1                  baseline
2                  baseline
3                  baseline
                ...        
345596    wasting_treatment
345597    wasting_treatment
345598    wasting_treatment
345599    wasting_treatment
Name: scenario, Length: 345600, dtype: category
Categories (3, object): ['baseline' < 'wasting_treatment' < 'sqlns']

In [43]:
vr.to_ordered_categoricals(df, inplace=True)
list(df.dtypes)

[dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 CategoricalDtype(categories=['baseline', 'wasting_treatment', 'sqlns'], ordered=True),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 CategoricalDtype(categories=['early_neonatal', 'late_neonatal', '1-5_months',
                   '6-11_months', '12_to_23_months', '2_to_4', 'all_ages'],
 , ordered=True)]

# Test whether categorical dtypes are preserved when computing ratios

It looks like **yes** if the Categorical is in a column (e.g. numerator_broadcast), but **no** if the Categorical is in the index (e.g. strata). Darn.

In [34]:
data2 = vr.VivariumResults.cleaned_from_model_spec('4.5.2')
data2.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'person_time',
 'cause_state_person_time',
 'cause_transition_count']

In [35]:
data2.to_ordered_categoricals()

In [36]:
data2.person_time.dtypes

age                category
input_draw            int64
measure              object
scenario           category
                     ...   
x_factor             object
x_factor_effect     float64
year                 object
value               float64
Length: 11, dtype: object

In [37]:
data2.wasting_state_person_time.dtypes

sex                    object
year                   object
wasting_state        category
measure                object
                       ...   
x_factor               object
sq_lns                 object
wasting_treatment      object
age                  category
Length: 12, dtype: object

In [38]:
wasting_prevalence = vr.get_prevalence(data2, 'wasting', ['age', 'sex', 'year'])
wasting_prevalence

,age,sex,year,input_draw,scenario,wasting_state,value,numerator_measure,denominator_measure,multiplier
0,1-5_months,female,2022,29,baseline,severe_acute_malnutrition,0.023122,state_person_time,person_time,1
1,1-5_months,female,2022,29,baseline,moderate_acute_malnutrition,0.080510,state_person_time,person_time,1
2,1-5_months,female,2022,29,baseline,mild_child_wasting,0.197947,state_person_time,person_time,1
3,1-5_months,female,2022,29,baseline,susceptible_to_child_wasting,0.698422,state_person_time,person_time,1
...,...,...,...,...,...,...,...,...,...,...
8636,late_neonatal,male,2026,946,wasting_treatment,severe_acute_malnutrition,0.041756,state_person_time,person_time,1
8637,late_neonatal,male,2026,946,wasting_treatment,moderate_acute_malnutrition,0.099042,state_person_time,person_time,1
8638,late_neonatal,male,2026,946,wasting_treatment,mild_child_wasting,0.202377,state_person_time,person_time,1
8639,late_neonatal,male,2026,946,wasting_treatment,susceptible_to_child_wasting,0.656824,state_person_time,person_time,1


In [42]:
wasting_prevalence.dtypes.to_dict()

{'age': dtype('O'),
 'sex': dtype('O'),
 'year': dtype('O'),
 'input_draw': dtype('int64'),
 'scenario': dtype('O'),
 'wasting_state': CategoricalDtype(categories=['severe_acute_malnutrition', 'moderate_acute_malnutrition',
                   'mild_child_wasting', 'susceptible_to_child_wasting'],
 , ordered=True),
 'value': dtype('float64'),
 'numerator_measure': dtype('O'),
 'denominator_measure': dtype('O'),
 'multiplier': dtype('int64')}

In [45]:
wasting_prevalence.age.unique()

array(['1-5_months', '12_to_23_months', '2_to_4', '6-11_months',
       'early_neonatal', 'late_neonatal'], dtype=object)

In [47]:
wasting_prevalence.isna().any().any()

False

In [44]:
pd.concat([data.deaths, data.ylls, data.ylds])

,sex,year,cause,measure,input_draw,scenario,x_factor_effect,value,wasting_state,age
0,female,2022,diarrheal_diseases,death,29,baseline,1.1,0.000000,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,death,29,baseline,1.1,0.000000,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,death,29,baseline,1.1,0.000000,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,death,29,baseline,1.1,0.000000,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...,...
215996,male,2026,severe_acute_malnutrition,ylds,946,wasting_treatment,1.5,0.000000,susceptible_to_child_wasting,2_to_4
215997,male,2026,severe_acute_malnutrition,ylds,946,wasting_treatment,1.5,0.000000,mild_child_wasting,2_to_4
215998,male,2026,severe_acute_malnutrition,ylds,946,wasting_treatment,1.5,0.000000,moderate_acute_malnutrition,2_to_4
215999,male,2026,severe_acute_malnutrition,ylds,946,wasting_treatment,1.5,37.570543,severe_acute_malnutrition,2_to_4


# Test updated module with old model versions

In [48]:
vr.models

,model_id,model_name,run_id
0,2.3,v2.3_wasting_birth_prevalence,2021_07_26_17_14_31
1,2.4,v2.4_corrected_fertility,2021_08_03_15_08_32
2,2.5,v2.5_stunting,2021_08_05_16_17_12
3,3.0,v3.0_sq_lns,2021_08_16_17_54_19
4,3.1,v3.1_sq_lns_stunting_stratified,2021_08_24_10_28_32
5,4.0,v4.0_wasting_treatment,2021_09_20_14_45_25
6,4.1,v4.1_wasting_treatment,2021_09_24_16_36_30
7,4.5.2,v4.5.2_x_factor,2021_09_29_12_12_47


In [49]:
results = {model_id: vr.VivariumResults.cleaned_from_model_spec(model_id) for model_id in vr.models.model_id}
results.keys()

dict_keys(['2.3', '2.4', '2.5', '3.0', '3.1', '4.0', '4.1', '4.5.2'])

In [52]:
results['2.3'].table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'person_time',
 'cause_state_person_time',
 'cause_transition_count']

In [60]:
results['2.5'].table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'person_time',
 'cause_state_person_time',
 'cause_transition_count']

In [55]:
results['3.1'].table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'person_time',
 'cause_state_person_time',
 'cause_transition_count']

In [56]:
results['3.1'].person_time

,age,input_draw,measure,scenario,sex,sq_lns,year,value
0,1-5_months,29,person_time,baseline,female,covered,2022,0.000000
1,1-5_months,29,person_time,baseline,female,covered,2023,0.000000
2,1-5_months,29,person_time,baseline,female,covered,2024,0.000000
3,1-5_months,29,person_time,baseline,female,covered,2025,0.000000
...,...,...,...,...,...,...,...,...
2876,late_neonatal,946,person_time,treatment_and_prevention,male,uncovered,2023,6263.872690
2877,late_neonatal,946,person_time,treatment_and_prevention,male,uncovered,2024,6284.019165
2878,late_neonatal,946,person_time,treatment_and_prevention,male,uncovered,2025,6291.353867
2879,late_neonatal,946,person_time,treatment_and_prevention,male,uncovered,2026,6240.344969


In [57]:
results['4.1'].person_time

,age,input_draw,measure,scenario,sex,sq_lns,wasting_treatment,year,value
0,1-5_months,29,person_time,baseline,female,covered,covered,2022,0.000000
1,1-5_months,29,person_time,baseline,female,covered,covered,2023,0.000000
2,1-5_months,29,person_time,baseline,female,covered,covered,2024,0.000000
3,1-5_months,29,person_time,baseline,female,covered,covered,2025,0.000000
...,...,...,...,...,...,...,...,...,...
8636,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,2023,625.863107
8637,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,2024,629.230664
8638,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,2025,628.206708
8639,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,2026,623.451061


In [58]:
results['4.5.2'].person_time

,age,input_draw,measure,scenario,sex,sq_lns,wasting_treatment,x_factor,x_factor_effect,year,value
0,1-5_months,29,person_time,baseline,female,covered,covered,cat1,1.1,2022,0.000000
1,1-5_months,29,person_time,baseline,female,covered,covered,cat1,1.1,2023,0.000000
2,1-5_months,29,person_time,baseline,female,covered,covered,cat1,1.1,2024,0.000000
3,1-5_months,29,person_time,baseline,female,covered,covered,cat1,1.1,2025,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
86396,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,cat2,1.5,2023,50.591376
86397,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,cat2,1.5,2024,52.776181
86398,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,cat2,1.5,2025,52.197125
86399,late_neonatal,946,person_time,wasting_treatment,male,uncovered,uncovered,cat2,1.5,2026,49.886379


In [59]:
results['4.5.2'].wasting_transition_count

,sex,year,measure,input_draw,scenario,x_factor_effect,value,x_factor,sq_lns,wasting_treatment,age,transition,from_state,to_state
0,female,2022,transition_count,29,baseline,1.1,0.0,cat2,covered,covered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
1,female,2022,transition_count,29,baseline,1.1,0.0,cat1,covered,covered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
2,female,2022,transition_count,29,baseline,1.1,0.0,cat2,covered,uncovered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
3,female,2022,transition_count,29,baseline,1.1,0.0,cat1,covered,uncovered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604796,male,2026,transition_count,946,wasting_treatment,1.5,1629.0,cat2,uncovered,covered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
604797,male,2026,transition_count,946,wasting_treatment,1.5,483.0,cat1,uncovered,covered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
604798,male,2026,transition_count,946,wasting_treatment,1.5,167.0,cat2,uncovered,uncovered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
604799,male,2026,transition_count,946,wasting_treatment,1.5,60.0,cat1,uncovered,uncovered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
